In [ ]:
import os
ROOT = "/content/drive/MyDrive/001_SUNNY_BHAVEEN_CHANDRA/001_My_Classes/DLCVNLP/Live Lectures Material DLCVNLP April 17th batch/ALL NOTES/Feb 06"

os.chdir(ROOT)

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [ ]:
def get_plot(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history[f"val_{metric}"])
  plt.xlabel("Epochs ->>")
  plt.ylabel(f"{metric} ->>")
  plt.legend([metric, f"val_{metric}"])

In [ ]:
dataset_name = "imdb_reviews"

dataset, info = tfds.load(dataset_name, with_info=True, as_supervised=True)

train_ds, test_ds = dataset["train"], dataset["test"]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU6C3NK/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU6C3NK/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# check the first batch -

for example, label in train_ds.take(1):
  print(f"test_example: \n{example.numpy()} \n")
  print(f"label: {label.numpy()}")

test_example: 
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it." 

label: 0


In [ ]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
class Config:
  BUFFER_SIZE = 10000
  BATCH_SIZE = 64
  VOCAB_SIZE = 1000
  OUTPUT_DIM = 64
  EPOCHS = 10
  BASE_LOG_DIR = "base_log_dir"
  TRAINED_MODEL_DIR = os.path.join(BASE_LOG_DIR, "models")
  CKPT_DIR = os.path.join(BASE_LOG_DIR, "checkpoints")
  TB_ROOT_LOG_DIR = os.path.join(BASE_LOG_DIR, "tb_log_dir")

In [ ]:
config = Config()

In [ ]:
config.BUFFER_SIZE

10000

In [ ]:
# shuffling and batching of the train dataset

train_ds = train_ds.shuffle(config.BUFFER_SIZE).batch(config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_ds.take(1):
  print(f"test_example: \n{example.numpy()[:3]} \n")
  print(f"label: {label.numpy()[:3]}")

test_example: 
[b'This movie sucked ! They took something from my childhood ,and raped it in an outhouse! This movie was so bad I wanted to go home and hold my "Dukes" dvds and cry in a corner. The cast was terrible ! It wasn\'t "The Dukes", it was Stiffler and Jackass driving a car. When was Boss Hogg evil? When was Rosco a tough guy? They never were ! Boss Hogg was greedy and Rosco was an idiot. When did Jesse smoke pot? He never did ! Now don\'t get me wrong,I\'m very liberal and there\'s nothing wrong with a little chiba, but it had no place in this movie! The only thing good about this movie was the trailers before the movie and the end credits. It was a waste of money time and air. Avoid at all costs!!!!!!!!'
 b"Ben a out-of-town cop is convinced his sister was brutally killed and wants to bring her killer to justice, but he's approached by Stefan who believes his sister was a victim of a werewolf cult. So Ben, his sister's best friend and Stefan travel to Transylvania to put a e

In [ ]:
# text encoder 

# keep the data as it is without custom standardisation
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=config.VOCAB_SIZE)

# adapt to change only on text data
encoder.adapt(train_ds.map(lambda text, label: text))


## text, label ## <--- train data is arranged

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [ ]:
print("text examples\n", example.numpy()[:3])

text examples
 [b'This movie sucked ! They took something from my childhood ,and raped it in an outhouse! This movie was so bad I wanted to go home and hold my "Dukes" dvds and cry in a corner. The cast was terrible ! It wasn\'t "The Dukes", it was Stiffler and Jackass driving a car. When was Boss Hogg evil? When was Rosco a tough guy? They never were ! Boss Hogg was greedy and Rosco was an idiot. When did Jesse smoke pot? He never did ! Now don\'t get me wrong,I\'m very liberal and there\'s nothing wrong with a little chiba, but it had no place in this movie! The only thing good about this movie was the trailers before the movie and the end credits. It was a waste of money time and air. Avoid at all costs!!!!!!!!'
 b"Ben a out-of-town cop is convinced his sister was brutally killed and wants to bring her killer to justice, but he's approached by Stefan who believes his sister was a victim of a werewolf cult. So Ben, his sister's best friend and Stefan travel to Transylvania to put a e

In [ ]:
encoder_example = encoder(example)[:3].numpy()
encoder_example

array([[11, 18,  1, ...,  0,  0,  0],
       [ 1,  4,  1, ...,  0,  0,  0],
       [ 1,  1,  6, ...,  0,  0,  0]])

In [ ]:
# model for BiRNN

embedding_layer = tf.keras.layers.Embedding(
    input_dim=len(encoder.get_vocabulary()),
    output_dim=config.OUTPUT_DIM,
    mask_zero=True # use masking to handle the variable seq lengths example <sos>, <pad>, <eod>
)

In [ ]:
LAYERS = [
          encoder,
          embedding_layer,
          tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
          tf.keras.layers.Dense(64, activation="relu"),
          tf.keras.layers.Dense(1)
]

In [ ]:
model = tf.keras.Sequential(LAYERS)

In [ ]:
[layer.supports_masking for layer in model.layers]

[False, True, True, True, True]

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

In [ ]:
import time

def callbacks(base_dir="."):
  # tb callbacks

  base_log_dir = config.TB_ROOT_LOG_DIR
  unqiue_log = time.asctime().replace(" ","_").replace(":", "")
  tb_log_dir = os.path.join(base_log_dir, unqiue_log)
  os.makedirs(tb_log_dir, exist_ok=True)

  tb_cb = tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)

  # ckpt callbacks
  ckpt_file = os.path.join(config.CKPT_DIR, "model")
  os.makedirs(config.CKPT_DIR, exist_ok=True)

  ckpt_cb = tf.keras.callbacks.ModelCheckpoint(
      filepath = ckpt_file, 
      save_best_only = True)
  
  callback_list = [tb_cb, ckpt_cb]

  return callback_list

In [ ]:
callback_list = callbacks()

In [ ]:
history = model.fit(train_ds, 
                    epochs=config.EPOCHS, 
                    validation_data=test_ds, 
                    validation_steps=30, 
                    callbacks=callback_list)

Epoch 1/10
380/391 [============================>.] - ETA: 21s - loss: 0.6464 - accuracy: 0.5641

KeyboardInterrupt: ignored

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

test_loss, test_acc

In [ ]:
get_plot(history=history, metric="accuracy")

In [ ]:
get_plot(history=history, metric="loss")

In [ ]:
%load_ext tensorboard

%tensorboard --logdir base_log_dir/tb_log_dir

In [ ]:
sample = ("The movie was cool. The animation and the graphics were of this world. I would recommend this movie")

In [ ]:
# def get_